In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def read_excel(path="./data.xlsx"):
    data_xls = pd.io.excel.ExcelFile(path)
    data={}
    print(data_xls.sheet_names)
    for name in data_xls.sheet_names:
        df=pd.read_excel(data_xls,sheet_name=name)
        data[name]=df
        # print(name, " : df = ",df , type(df))
        if df.empty:
            continue
    return data

data = read_excel()
data

In [ ]:
pinformation = data["驾驶员基本信息"]
pwork = data["工作时长"]
service_violation = data["服务违章"]
safety_violation = data["安全违章"]
safety_accident = data["安全事故"]

In [ ]:
# pinformation
# 男 - 1 ； 女 - 0
pinformation["性别"].value_counts()
pinformation.loc[pinformation["性别"]=='男' , '性别']=1
pinformation.loc[pinformation["性别"]=='女' , '性别']=0
pinformation

In [ ]:
pinformation["分公司"].value_counts()
pinformation = pinformation.drop(labels="分公司", axis=1)
pinformation

In [ ]:
pinformation["学历"].value_counts()
# 普高 0，中专 1， 初中 2， 大专 3， 高中（普高） 4， 本科 5， 技校 6， 职高 7 ，小学 8， nan 9
pinformation.loc[pinformation["学历"]=='普高' , '学历']=0
pinformation.loc[pinformation["学历"]=='中专' , '学历']=1
pinformation.loc[pinformation["学历"]=='初中' , '学历']=2
pinformation.loc[pinformation["学历"]=='大专' , '学历']=3
pinformation.loc[pinformation["学历"]=='高中（普高）' , '学历']=4
pinformation.loc[pinformation["学历"]=='本科' , '学历']=5
pinformation.loc[pinformation["学历"]=='技校' , '学历']=6
pinformation.loc[pinformation["学历"]=='职高' , '学历']=7
pinformation.loc[pinformation["学历"]=='小学' , '学历']=8
pinformation["学历"] = pinformation["学历"].fillna(9)
pinformation

In [ ]:
pinformation['出生日期'] = pd.to_datetime(pinformation['出生日期'])
import datetime as dt
now_year = dt.datetime.today().year
# pinformation['age'] = now_year - frame.birth.dt.year
pinformation['出生日期'] = now_year - pinformation['出生日期'].dt.year

pinformation['入职日期'] = pd.to_datetime(pinformation['入职日期'])
now_year = dt.datetime.today().year
# pinformation['age'] = now_year - frame.birth.dt.year
pinformation['入职日期'] = now_year - pinformation['入职日期'].dt.year

pinformation

In [ ]:
# pinformation = pinformation.drop(labels=["事故", "安全违章", "服务违章"], axis=1)


In [ ]:
safety_accident.columns = safety_accident.columns.str.replace('事故日期', '开班日期')

safety_accident['开班日期'] = pd.to_datetime(safety_accident['开班日期'],format='%Y-%M-%d').dt.date.astype(object)
safety_accident
pwork['开班日期'] = pd.to_datetime(pwork['开班日期'],format='%Y-%M-%d').dt.date.astype(object)
res = pd.merge(pwork, safety_accident, how='right', on=['员工编号', '开班日期'])
res
# print(safety_accident)

In [ ]:
import matplotlib.pyplot as plt
y_1 = res['工作时长（小时）']
y_2 = res['驾车时长（小时）']
y_3 = res['运营时长（小时）']
x = [i for i in range(len(y_1))]
plt.plot(x, y_1)
plt.plot(x, y_2)
plt.plot(x, y_3)
plt.show()

print(res)
import seaborn as sns
hres = res[["工作时长（小时）", "驾车时长（小时）", "运营时长（小时）"]]
plot=sns.heatmap(res)
plt.show()

In [ ]:
hh = pd.merge(pwork, safety_accident, how='left', on=['员工编号'])
hh.insert(loc=len(hh.columns.array), column="事故", value = 1)
hh.loc[hh['事故原因'].isna(), "事故"]  = 0
print(hh)
import seaborn as sns
hres = hh[["工作时长（小时）", "驾车时长（小时）", "运营时长（小时）", "事故"]]

plt.subplots(figsize=(20,15))
ax = plt.axes()
ax.set_title("Correlation Heatmap")
corr = hres.corr()
sns.heatmap(corr,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
plt.show()

In [ ]:
oth_res = pd.merge(pwork, safety_accident, how='left', on=['员工编号'])
print(oth_res)
oth_y_1 = oth_res[oth_res['开班日期_y'].isna()]['工作时长（小时）']
oth_y_2 = oth_res[oth_res['开班日期_y'].isna()]['驾车时长（小时）']
oth_y_3 = oth_res[oth_res['开班日期_y'].isna()]['运营时长（小时）']
oth_x = [i for i in range(len(oth_y_1))]
plt.subplot(3, 2, 1)
plt.scatter(x, y_1)
plt.subplot(3, 2, 2)
plt.scatter(oth_x, oth_y_1)
plt.show()
plt.subplot(3, 2, 3)
plt.scatter(x, y_2)
plt.subplot(3, 2, 4)
plt.scatter(oth_x, oth_y_2)
plt.show()
plt.subplot(3, 2, 5)
plt.scatter(x, y_3)
plt.subplot(3, 2, 6)
plt.scatter(oth_x, oth_y_3)
plt.show()

In [ ]:
print(service_violation.groupby(service_violation['服务违章项目']).value_counts())

In [ ]:
service_res = pd.merge(service_violation, safety_accident, how='left', on=['员工编号'])
print(service_res)
no_num = len(service_res[oth_res['开班日期_y'].isna()])
ys_num = len(service_res['服务违章项目']) - no_num
plt.bar([0, 1], [no_num, ys_num],  width=0.5)
plt.show()
plt.pie([float(no_num)/float(no_num + ys_num), float(ys_num)/float(no_num + ys_num)])
plt.show()

In [ ]:
# pinformation
# pinformation = pinformation.drop(labels=[ "服务违章"], axis=1)
pinformation.insert(loc=6, column="服务违章", value = 0)

pinformation.loc[pinformation["员工编号"].isin(service_violation["员工编号"]), "服务违章"] = 1
pinformation

In [ ]:
safety_res = pd.merge(safety_violation, safety_accident, how='left', on=['员工编号'])
print(safety_res)
sno_num = len(safety_res[safety_res['开班日期'].isna()])
sys_num = len(safety_res['安全违章项目']) - sno_num
plt.bar([0, 1], [sno_num, sys_num],  width=0.5)
plt.show()
plt.pie([float(sno_num)/float(sno_num + sys_num), float(sys_num)/float(sno_num + sys_num)])
plt.show()

In [ ]:

pinformation.insert(loc=7, column="安全违章", value = 0)
pinformation
pinformation.loc[pinformation["员工编号"].isin(safety_violation["员工编号"]), "安全违章"] = 1
print(pinformation["安全违章"].value_counts())

In [ ]:
pinformation.insert(loc=8, column="事故", value = 0)
pinformation
pinformation.loc[pinformation["员工编号"].isin(safety_accident["员工编号"]), "事故"] = 1
pinformation

In [ ]:
pinformation = pinformation.drop(labels=["初领证日期"], axis=1)
pinformation = pinformation.drop(labels=["员工编号"], axis=1)
# pinformation = pinformation.drop(labels=["入职日期"], axis=1)
pinformation
print(pinformation['事故'].value_counts())
pinformation.to_csv("./data1.csv", index = False)

In [ ]:
print(pinformation[pinformation['安全违章']==1]['事故'].value_counts())

In [ ]:
data_list = pinformation.values.tolist()
print(data_list)
label_list = []
for i in range(len(data_list)):
    label_list.append([data_list[i][-1]])
    data_list[i] = data_list[i][:-1]
print(data_list)
print(label_list)

In [ ]:
from sklearn.metrics import precision_recall_curve, roc_curve, classification_report
def num_get(y_score,y_test):
    #由于我们是多标签模型，所以循环输出每种标签的值
    Precision,Recall,_ = precision_recall_curve(y_test[:],y_score[:])
    FPR,TPR,_ = roc_curve(y_test[:], y_score[:])
    return Precision,Recall,TPR,FPR

def draw_line(Precision,Recall,TPR,FPR,classes,title='model'):
    plt.figure(figsize=(14, 8))
    plt.suptitle(title+' 的P-R曲线和ROC曲线', fontsize = 30)
    ax1=plt.subplot(1,2,1)
    plt.xlabel("Recall")
    plt.ylabel("Preccision")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    l, = plt.plot(Recall, Precision)
    ax2=plt.subplot(1,2,2)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])

    l, = plt.plot(FPR, TPR)
    ax1.set_title("P-R Pic")
    ax2.set_title("ROC Pic")
    plt.savefig(title+'PR-ROC.png')
    plt.show()

In [ ]:
from sklearn import tree
import numpy as np
from io import StringIO
import pydotplus

X = np.array(data_list)
y = np.array(label_list)
tree_model = tree.DecisionTreeClassifier(criterion='gini',
                                         max_depth=None,
                                         min_samples_leaf=1,
                                         ccp_alpha=0.0)
tree_model.fit(X, y)
dot_data = StringIO()
feature_names = ["sex", "age", "grade", "word_year", "service_violation", "safe_violation"]
target_names = ['no_safe_accident', 'safe_accident']
tree.export_graphviz(tree_model,
                     out_file=dot_data,
                     feature_names=feature_names,
                     class_names=target_names,
                     filled=True,
                     rounded=True,
                     special_characters=True)
dot_data_val = dot_data.getvalue()
dot_data_val = dot_data_val.replace('helvetica', 'MicrosoftYaHei')
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
from IPython.display import Image  # 可视化展示决策树

Image(graph.create_png())